# Дашборд Кейса №2

## Импорты библиотек

In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr  # new
import os  # new


## Вспомогательные функции

Подгрузка csv файла

In [94]:
#подгрузка данных
def load_data(file):
    """
    Загружает CSV-файл в DataFrame и возвращает данные, их фрагмент и информационные сообщения.

    Функция предназначена для использования в интерактивных приложениях (например, Gradio),
    где пользователь загружает файл через интерфейс. Проверяет наличие файла, читает его
    с автоматическим парсингом дат, и возвращает структурированные данные и текстовые
    сообщения об успешной загрузке и объёме данных.

    Параметры
    ----------
    file : UploadedFile или None
        Объект загруженного файла. Должен иметь атрибут `name`, указывающий путь к файлу.
        Если значение `None`, функция возвращает сообщение об ошибке.

    Возвращает
    -------
    tuple
        Кортеж из четырёх элементов:
        - pd.DataFrame или None:
            Полный DataFrame с данными из CSV. Если файл не загружен — `None`.
        - pd.DataFrame:
            Первые 10 строк данных (`df.head(10)`) для предварительного просмотра.
            При отсутствии файла возвращается пустой DataFrame.
        - str:
            Сообщение о статусе загрузки: успех или ошибка.
        - str или None:
            Дополнительное описание содержимого файла: количество строк и столбцов.
            Если файл не загружен — возвращается `None`.

    Обработка данных
    ----------------
    - Чтение выполняется через `pd.read_csv()` по пути `file.name`.
    - Столбцы `survey_creation_dt` и `survey_response_dt` автоматически преобразуются в тип `datetime`.
    - Имя файла извлекается с помощью `os.path.basename()` для чистоты отображения.

    Примеры использования
    ---------------------
    >>> data, preview, status_msg, desc_msg = load_data(uploaded_file)
    >>> if data is not None:
    ...     st.success(status_msg)
    ...     st.write(desc_msg)
    ...     st.dataframe(preview)
    ... else:
    ...     st.error(status_msg)

    Замечания
    ---------
    - Если указанные столбцы с датами отсутствуют в CSV, pandas выдаст предупреждение.
    - Рекомендуется обернуть `pd.read_csv()` в блок try-except в продвинутых сценариях
      для обработки повреждённых или некорректных файлов.
    - Поддерживает работу только с CSV-файлами. Другие форматы не обрабатываются.
    """
    if file is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате csv", None
    df = pd.read_csv(
        file.name, parse_dates=["survey_creation_dt", "survey_response_dt"]
    )
    filename = os.path.basename(file.name)
    text_status = "Файл " + str(filename) + " успешно загружен"
    text_description = (
        "Файл содержит "
        + str(len(df))
        + " строк и "
        + str(len(df.columns))
        + " столбцов."
    )

    return df, df.head(10), text_status, text_description

Очистка данных

In [95]:
#Очистка данных
def clear_data(df):
    """
    Выполняет предобработку и очистку данных анкетного опроса.

    Функция принимает DataFrame, содержащий данные с датами создания и ответа на опрос,
    и выполняет следующие операции:
    1. Вычисляет время ответа в днях.
    2. Исправляет строки, где дата ответа раньше даты создания (путём перестановки дат).
    3. Заполняет пропущенные значения в столбце языка по умолчанию ('RU').
    Возвращает обновлённый DataFrame, его фрагмент и текстовые сообщения о ходе обработки.

    Параметры
    ----------
    df : pd.DataFrame или None
        Исходный DataFrame с данными опроса. Должен содержать столбцы:
        - 'survey_creation_dt' — дата и время создания опроса;
        - 'survey_response_dt' — дата и время ответа на опрос;
        - 'language' — язык, на котором заполнен опрос.
        Если передано значение `None`, функция возвращает сообщение об ошибке.

    Возвращает
    -------
    tuple
        Кортеж из четырёх элементов:
        - pd.DataFrame или None:
            Очищенный DataFrame с обновлёнными данными. При `df is None` возвращает `None`.
        - pd.DataFrame:
            Первые 10 строк очищенного DataFrame (`df.head(10)`) для предварительного просмотра.
        - str:
            Краткое сообщение о статусе обработки (успех или ошибка).
        - str или None:
            Подробное текстовое описание выполненных действий, включая статистику:
            количество записей с отрицательным временем, исправленных строк,
            заполненных пропусков и итоговый размер данных.
            При отсутствии входного DataFrame возвращается `None`.

    Этапы обработки
    ---------------
    1. **Расчёт времени ответа**:
       - Вычисляется разница между датами ответа и создания в секундах, затем переводится в дни.
       - Результат сохраняется в новом столбце `response_time_days`.

    2. **Исправление некорректных дат**:
       - Определяются строки, где `response_time_days < 0` (ответ раньше создания).
       - Формируется флаг `dates_swapped`.
       - Для таких строк даты создания и ответа меняются местами.
       - Время ответа пересчитывается.

    3. **Заполнение пропусков**:
       - Подсчитывается количество пропущенных значений в столбце `language`.
       - Все пропуски заменяются на значение 'RU' (по умолчанию).

    Примеры использования
    ---------------------
    >>> cleaned_df, preview, status, desc = clear_data(raw_df)
    >>> if cleaned_df is not None:
    ...     print(status)
    ...     print(desc)
    ... else:
    ...     print("Ошибка: данные не загружены")

    Замечания
    ---------
    - Требует, чтобы столбцы `survey_creation_dt` и `survey_response_dt` были типа `datetime64`.
    - Операция перестановки дат использует `.values` для корректной вставки при обмене.
    - Рекомендуется вызывать после `load_data()` или аналогичной функции загрузки.
    - Не выбрасывает исключения: при ошибках в данных (например, не-дата) поведение зависит от pandas.
      Для надёжности можно добавить обработку исключений.
    """
    if df is None:
        return None, pd.DataFrame(), "Ошибка. Загрузите файл в формате CSV.", None

    # 1. Вычисление времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    # 2. Исправление перепутанных дат (где ответ раньше создания)
    negative_before = (df["response_time_days"] < 0).sum()
    text_description = f"Записей с отрицательным временем ответа: {negative_before}.\n"

    df["dates_swapped"] = df["response_time_days"] < 0
    swap = df["dates_swapped"]
    df.loc[swap, ["survey_creation_dt", "survey_response_dt"]] = df.loc[
        swap, ["survey_response_dt", "survey_creation_dt"]
    ].values

    # Пересчёт времени ответа
    df["response_time_days"] = (
        df["survey_response_dt"] - df["survey_creation_dt"]
    ).dt.total_seconds() / (24 * 3600)

    negative_after = (df["response_time_days"] < 0).sum()
    text_description = text_description + (f"После исправления: {negative_after}. \n")

    # 3. Заполнение пропусков в language
    language_missing = df["language"].isnull().sum()
    df["language"] = df["language"].fillna("RU")
    text_description = text_description + (
        f"Заполнено пропусков в language: {language_missing}. \n"
    )

    text_status = "Файл успешно обработан"
    text_description = text_description + (
        "Актуальный файл содержит "
        + str(len(df))
        + " строк и "
        + str(len(df.columns))
        + " столбцов."
    )

    return df, df.head(10), text_status, text_description


Функция создания

In [96]:
def update_col_dropdown(df):
    if df is None:
        return gr.Dropdown(choices=[], value=None), "Нет данных: сначала загрузите CSV"
    cols = list(df.columns)
    return gr.Dropdown(choices=cols, value=(cols[0] if cols else None)), f"Колонки: {cols}"

In [97]:
def plot_df(df, col, top_k, mode):
    if df is None or col is None or col not in df.columns:
        return pd.DataFrame({"category": [], "value": []})

    s = df[col].astype(str).fillna("NaN")
    if mode == "share":
        vc = s.value_counts(normalize=True, dropna=False)  # доли [page:1]
    else:
        vc = s.value_counts(dropna=False)                  # частоты [page:1]

    vc = vc.head(int(top_k))
    return vc.rename_axis("category").reset_index(name="value")


In [98]:
def update_col_dropdown_with_stats(df):
    if df is None:
        return gr.Dropdown(choices=[], value=None), "Нет данных"

    # число уникальных по каждому столбцу [web:192]
    u = df.nunique(dropna=False).sort_values(ascending=False)

    # строки для выпадающего списка
    choices = [f"{col} ({int(cnt)})" for col, cnt in u.items()]
    value = choices[0] if choices else None
    return gr.Dropdown(choices=choices, value=value), f"Столбцов: {len(choices)}"

def _parse_choice(choice_text: str):
    # "language (12)" -> "language"
    if choice_text is None:
        return None
    return choice_text.rsplit(" (", 1)[0]

def cat_counts_for_choice(df, choice_text, top_k, mode):
    col = _parse_choice(choice_text)
    if df is None or col is None or col not in df.columns:
        return pd.DataFrame({"category": [], "value": []})

    s = df[col].astype(str).fillna("NaN")
    if mode == "share":
        vc = s.value_counts(normalize=True, dropna=False)  # доли [web:95]
    else:
        vc = s.value_counts(dropna=False)                  # частоты [web:95]

    vc = vc.head(int(top_k))
    return vc.rename_axis("category").reset_index(name="value")


## Структура дашборда

In [ ]:
# структура Дашборда
with gr.Blocks(title="Кейс 2 - Команда 8") as case2:
    gr.Markdown("""
    <div style="text-align:center">

    # Дашборд команды №8  
    Кейс №2. Пенсионный фонд, предоставляющий клиентам цифровые сервисы.

    </div>
    """)
    # df_state для того, чтобы сохранить данные для кнопок
    df_state = gr.State(None)

    # структура дашборда
    with gr.Tabs():
        # Таб с загрузкой данных
        with gr.Tab("Загрузка данных"):
            with gr.Row():
                result_box = gr.Textbox(label="Статус", lines = 3, max_lines= 15, scale = 1)
                description_box = gr.Textbox(label="Описание", lines = 3, max_lines = 15, scale = 3)
            file_input = gr.File(label="Выберите CSV-файл", file_types=[".csv"], scale = 1, height = 100)
            with gr.Row():
                load_button = gr.Button("Загрузить файл csv")
                clear_button = gr.Button("Очистить данные")
            head_df = gr.DataFrame(label="Пример данных", type="pandas", interactive=False, 
                                   column_widths=[120, 160, 160, 70, 70, 70, 70, 90, 110, 110, 110],
                                   wrap = True
                                   )
        # Таб с авторами
        with gr.Tab("Авторы"):
            # Автор 1
            with gr.Row():
                gr.Image(value="team/team_01.png", height=250, width=50)
                gr.Image(value="team/team_02.png", height=250, width=50)
                gr.Image(value="team/team_03.png", height=250, width=50)
            with gr.Row():
                gr.Markdown(
                        """
        **Имя Фамилия 1**  
        Роль: Роль №1

        [LinkedIn](https://linkedin.com/in/author1)  
        [GitHub](https://github.com/author1)
                        """
                    )
                gr.Markdown(
                        """
        **Имя Фамилия 2**  
        Роль: Роль №2

        [Telegram](https://t.me/author2)  
        [GitHub](https://github.com/author2)
                        """
                    )
                gr.Markdown(
                        """
        **Имя Фамилия 3**  
        Роль: Роль №3

        [VK](https://vk.com/author3)  
        [GitHub](https://github.com/author3)
                        """
                    )
        #таб с графиками
        with gr.Tab("Графики"):
            with gr.Row():
                col = gr.Dropdown(label="Столбец", choices=[], value=None, interactive=True)
                top_k = gr.Slider(3, 50, value=15, step=1, label="Top-K категорий")
                mode = gr.Radio(choices=["count", "share"], value="count", label="Режим")
            #dbg = gr.Textbox(label="Debug", lines=2)

            plot = gr.BarPlot(
                x="category",
                y="value",
                y_title="count/share",
                x_title="Категория",
                title="Распределение по столбцу",
            )


                

# после загрузки CSV -> обновить dropdown со статистикой
    load_evt = load_button.click(load_data, inputs=[file_input],
                                outputs=[df_state, head_df, result_box, description_box])
    load_evt.then(update_col_dropdown_with_stats, inputs=[df_state], outputs=[col, dbg])

    # после очистки -> очистить dropdown
    clear_evt = clear_button.click(clear_data, inputs=[df_state],
                                outputs=[df_state, head_df, result_box, description_box])
    clear_evt.then(update_col_dropdown_with_stats, inputs=[df_state], outputs=[col, dbg])

    # график: теперь inputs=[df_state, col, ...], где col содержит "имя (N)"
    col.change(cat_counts_for_choice, inputs=[df_state, col, top_k, mode], outputs=plot)
    top_k.change(cat_counts_for_choice, inputs=[df_state, col, top_k, mode], outputs=plot)
    mode.change(cat_counts_for_choice, inputs=[df_state, col, top_k, mode], outputs=plot)



In [100]:
case2.launch(share=False, inline=True)

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
